In [1]:
import os
import pandas as pd
import re

In [2]:
project_root = "D:/Projects/Summer-2024-ECE-597-Group8"
normal_sample_numbers = 2571 * 10

In [3]:
def normal_extract_subject_body(text):
    # Extract the subject using a regular expression
    subject_match = re.search(r"Subject: (.*)", text)
    subject = subject_match.group(1) if subject_match else "Subject Not Found"

    # Clean the text by removing everything from "Message-ID:" up to "X-FileName:"
    cleaned_text = re.sub(r"Message-ID:.*?X-FileName:.*?\n", "", text, flags=re.S)

    return subject, cleaned_text

In [4]:
phishing_data_path = os.path.join(
    project_root, "data", "raw", "CaptstoneProjectData_2024.csv"
)
normal_data_path = os.path.join(
    project_root, "data", "raw", "EnronEmailDataset.csv"
)

phishing_data = pd.read_csv(phishing_data_path)
normal_data = pd.read_csv(
    normal_data_path, nrows=normal_sample_numbers
)

phishing_data.drop(columns=["Unnamed: 2", "Unnamed: 3"], inplace=True)
phishing_data["Subject"] = phishing_data["Subject"].fillna("Subject Not Found")
phishing_data.dropna(
    subset=["Body"], inplace=True
)
phishing_data.reset_index(drop=True, inplace=True)
normal_data[["Subject", "Body"]] = normal_data["message"].apply(
    lambda x: pd.Series(normal_extract_subject_body(x))
)

phishing_data["Label"] = 1
normal_data["Label"] = 0

In [5]:
df_data = pd.concat([phishing_data, normal_data], ignore_index=True)
df_data.info()
df_labels = df_data["Label"]
df_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28281 entries, 0 to 28280
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Subject  28281 non-null  object
 1   Body     28281 non-null  object
 2   Label    28281 non-null  int64 
 3   file     25710 non-null  object
 4   message  25710 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.1+ MB
<class 'pandas.core.series.Series'>
RangeIndex: 28281 entries, 0 to 28280
Series name: Label
Non-Null Count  Dtype
--------------  -----
28281 non-null  int64
dtypes: int64(1)
memory usage: 221.1 KB


In [6]:
from get_email_length import fetch_email_length

df_wordCount = fetch_email_length(df_data.copy())
df_wordCount.info()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xfy53\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xfy53\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\xfy53\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\xfy53\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                                 Subject  \
0      ®Review your shipment details / Shipment Notif...   
1                                Υоur ассоunt іѕ оn hоld   
2      Completed: Invoice # KZ89TYS2564  +1.813.776.1410   
3                                 UVic IMPORTANT NOTICE!   
4              You have (6) Suspended incoming messages    
...                                                  ...   
28276    Out of the Office - Afternoon of Friday, June 9   
28277                                    Re: TDS project   
28278                              Global Systems Matrix   
28279             Mid Year Energy Operations PRC Meeting   
28280                             Mgmt Summary & HotList   

                                                    Body  Label  \
0       [] Dear , An agent of ours tried to ship a pa...      1   
1       Votre réponse a bien été prise en compte. [] ...      1   
2               [DocuSign] [] Your document has been ...      1   
3      Your

In [7]:
from homoglyphs import feature_homoglyphs

df_homoglyphs = feature_homoglyphs(df_data.copy())
df_homoglyphs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28281 entries, 0 to 28280
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Homoglyphs  28281 non-null  int64
dtypes: int64(1)
memory usage: 221.1 KB


In [8]:
from abnormal_number_extract import extract_abnormal_number

df_abnormal = extract_abnormal_number(df_data.copy())
df_abnormal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28281 entries, 0 to 28280
Data columns (total 1 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Total_Abnormal_Count  28281 non-null  int64
dtypes: int64(1)
memory usage: 221.1 KB


In [9]:
from html_JS import extract_features_from_message

df_specials = df_data.copy().apply(
    lambda row: extract_features_from_message(
        str(row['Subject']) + ' ' + str(row['Body'])
    ), axis=1
)
df_specials = pd.DataFrame(df_specials.tolist(), columns=['html_tags', 'js_code'])
df_specials.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28281 entries, 0 to 28280
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   html_tags  28281 non-null  int64
 1   js_code    28281 non-null  int64
dtypes: int64(2)
memory usage: 442.0 KB


In [10]:
from phishing_bow import feature_extract_bow

df_bow = feature_extract_bow(df_data.copy())
df_bow.info()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xfy53\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xfy53\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\xfy53\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\xfy53\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28281 entries, 0 to 28280
Columns: 187 entries, bow_able to bow_year
dtypes: int64(187)
memory usage: 40.3 MB


In [11]:
from phishing_tfidf import feature_extract_tfidf

df_tfidf = feature_extract_tfidf(df_data.copy())
df_tfidf.info()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xfy53\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xfy53\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\xfy53\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\xfy53\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28281 entries, 0 to 28280
Columns: 187 entries, tfidf_able to tfidf_year
dtypes: float64(187)
memory usage: 40.3 MB


In [12]:
df_features = pd.concat([df_wordCount, df_homoglyphs, df_abnormal, df_specials], axis=1)

df_features_labels = pd.concat([df_features, df_labels], axis=1)
df_bow_labels = pd.concat([df_bow, df_labels], axis=1)
df_tfidf_labels = pd.concat([df_tfidf, df_labels], axis=1)
df_features_bow_labels = pd.concat([df_features, df_bow, df_labels], axis=1)
df_features_tfidf_labels = pd.concat([df_features, df_tfidf, df_labels], axis=1)

In [13]:
df_features_labels = df_features_labels.sample(frac=1, random_state=42).reset_index(drop=True)
df_bow_labels = df_bow_labels.sample(frac=1, random_state=42).reset_index(drop=True)
df_tfidf_labels = df_tfidf_labels.sample(frac=1, random_state=42).reset_index(drop=True)
df_features_bow_labels = df_features_bow_labels.sample(frac=1, random_state=42).reset_index(drop=True)
df_features_tfidf_labels = df_features_tfidf_labels.sample(frac=1, random_state=42).reset_index(drop=True)


In [14]:
# save the data
df_features_labels.to_csv(os.path.join(project_root, "data", "processed", "features_labels.csv"), index=False)
df_bow_labels.to_csv(os.path.join(project_root, "data", "processed", "bow_labels.csv"), index=False)
df_tfidf_labels.to_csv(os.path.join(project_root, "data", "processed", "tfidf_labels.csv"), index=False)
df_features_bow_labels.to_csv(os.path.join(project_root, "data", "processed", "features_bow_labels.csv"), index=False)
df_features_tfidf_labels.to_csv(os.path.join(project_root, "data", "processed", "features_tfidf_labels.csv"), index=False)